In [1]:
# from src.dataset.process import process_data_pandas
# # data_file = "data/mini_sparkify_event_data.json"
# # data_file = "data/mini_sparkify_event_data_sample.json"
# data_file = "data/mini_sparkify_event_data.parquet/"
# gather_col = "userId"
# sort_col = "ts"
# alias = "data"
# target_col = "page"
# exploded_col = "explode"
# emb_out_col = "embeddings"
# target_out_col = "targets"
# process_data_pandas(
#     parquet_file=data_file,
#     gather_col=gather_col,
#     sort_col=sort_col,
#     target_col=target_col,
#     cache_size=1000*5,
#     device="cuda",
# )

In [2]:
from src.dataset.dataset import SparkifyDataset

In [3]:
train_file = "data/train_dataset"

In [4]:
ds = SparkifyDataset(train_file)

In [8]:
from src.preprocess.collate_fn import mat_collate_fn

In [9]:
from torch.utils.data import DataLoader

In [10]:
dl = DataLoader(ds, batch_size=2, shuffle=True, collate_fn=mat_collate_fn)

In [12]:
emb, pos_indices, targets, masks = next(iter(dl))

In [13]:
emb.shape

torch.Size([2, 128, 384])